In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install turicreate
import turicreate as tc

     |████████████████████████████████| 91.9 MB 3.8 kB/s  eta 0:00:01
     |████████████████████████████████| 3.5 MB 52.8 MB/s eta 0:00:01
     |████████████████████████████████| 322 kB 39.5 MB/s eta 0:00:01
     |████████████████████████████████| 86.3 MB 125 kB/s  eta 0:00:01
     |████████████████████████████████| 50 kB 3.6 MB/s  eta 0:00:01
     |████████████████████████████████| 449 kB 38.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 54.5 MB/s eta 0:00:01
  Created wheel for resampy: filename=resampy-0.2.1-py3-none-any.whl size=320848 sha256=0cbffd5c66e6171cc8d33cb12244d5e805a6aabb61aae9d5517f6460a8e6a51a
  Stored in directory: /root/.cache/pip/wheels/71/74/53/d5ceb7c5ee7a168c7d106041863e71ac3273f4a4677743a284
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7539 sha256=ddcc0ecc1f5482f95b14fac254d021f5f27406e8f0b12661eb64b1c591094e6d
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Succ

    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow-probability 0.11.0 requires gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
librosa 0.8.0 requires resampy>=0.2.2, but you'll have resampy 0.2.1 which is incompatible.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m p

We will use LASSO to select features, building on a pre-implemented solver for LASSO 
1.Run LASSO with different L1 penalties.
2.Choose best L1 penalty using a validation set.
3.Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In [2]:
sales= tc.SFrame('../input/house-data')

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

In [5]:
model_all = tc.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.424820     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 1.559983     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [8]:
# non_zero_weight = model_all.get("coefficients")["value"]
non_zero_weight = model_all.coefficients[model_all.coefficients["value"] > 0]
non_zero_weight.print_rows(num_rows=20)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 274873.0559504957  |  None  |
|    bathrooms     |  None | 8468.531086910072  |  None  |
|   sqft_living    |  None | 24.420720982445214 |  None  |
| sqft_living_sqrt |  None | 350.0605533860648  |  None  |
|      grade       |  None | 842.0680348976282  |  None  |
|    sqft_above    |  None | 20.024722417091304 |  None  |
+------------------+-------+--------------------+--------+
[6 rows x 4 columns]



To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:

1.Split our sales data into 2 sets: training and test
2.Further split our training data into two sets: train, validation
3.Be very careful that you use seed = 1 to ensure you get the same answer!

In [9]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

In [10]:
import numpy as np
import pprint 

validation_rss = {}
for l1_penalty in np.logspace(1, 7, num=13):
    model = tc.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    predictions = model.predict(validation)
    residuals = validation['price'] - predictions
    rss = sum(residuals**2)
    validation_rss[l1_penalty] = rss

# pprint.pprint(result_dict)
print (min(validation_rss.items(), key=lambda x: x[1]) )

(10.0, 625766285142461.2)


In [11]:
model_test = tc.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=10.0)
predictions_test = model.predict(testing)
residuals_test = testing['price'] - predictions_test
rss_test = sum(residuals_test**2)
print (rss_test)

156972779668688.7


In [12]:
non_zero_weight_test = model_test.coefficients[model_test.coefficients["value"] > 0]
print (model_test.coefficients["value"].nnz())
non_zero_weight_test.print_rows(num_rows=20)

18
+------------------+-------+----------------------+--------+
|       name       | index |        value         | stderr |
+------------------+-------+----------------------+--------+
|   (intercept)    |  None |  18993.427212770577  |  None  |
|     bedrooms     |  None |  7936.967679031306   |  None  |
| bedrooms_square  |  None |   936.993368193299   |  None  |
|    bathrooms     |  None |  25409.58893412067   |  None  |
|   sqft_living    |  None |  39.11513637970762   |  None  |
| sqft_living_sqrt |  None |  1124.6502128077207  |  None  |
|     sqft_lot     |  None | 0.003483618222989743 |  None  |
|  sqft_lot_sqrt   |  None |  148.2583910114082   |  None  |
|      floors      |  None |  21204.33546695013   |  None  |
|  floors_square   |  None |  12915.524336072436  |  None  |
|    waterfront    |  None |  601905.5945452718   |  None  |
|       view       |  None |  93312.85731187189   |  None  |
|    condition     |  None |  6609.035712447216   |  None  |
|      grade       | 

# Limit the number of nonzero weights
What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:

Explore a large range of l1_penalty values to find a narrow region of l1_penalty values where models are likely to have the desired number of non-zero weights.
Further explore the narrow region you found to find a good value for l1_penalty that achieves the desired sparsity. Here, we will again use a validation set to choose the best value for l1_penalty.

In [13]:
max_nonzeros = 7

Exploring the larger range of values to find a narrow range with the desired sparsity
Let's define a wide range of possible l1_penalty_values:

In [14]:
l1_penalty_values = np.logspace(8, 10, num=20)
print (l1_penalty_values)

[1.00000000e+08 1.27427499e+08 1.62377674e+08 2.06913808e+08
 2.63665090e+08 3.35981829e+08 4.28133240e+08 5.45559478e+08
 6.95192796e+08 8.85866790e+08 1.12883789e+09 1.43844989e+09
 1.83298071e+09 2.33572147e+09 2.97635144e+09 3.79269019e+09
 4.83293024e+09 6.15848211e+09 7.84759970e+09 1.00000000e+10]


Now, implement a loop that search through this space of possible l1_penalty values:

For l1_penalty in np.logspace(8, 10, num=20):
Fit a regression model with a given l1_penalty on TRAIN data. Specify l1_penalty=l1_penalty and l2_penalty=0. in the parameter list. When you call linear_regression.create() make sure you set validation_set = None
Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
Hint: model['coefficients']['value'] gives you an SArray with the parameters you learned. If you call the method .nnz() on it, you will find the number of non-zero parameters!

In [16]:
coef_dict = {}
for l1_penalty in l1_penalty_values:
    model = tc.linear_regression.create(training, target ='price', features=all_features,
                                              validation_set=None, verbose=None,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    coef_dict[l1_penalty] = model.coefficients['value'].nnz()

In [17]:
pprint.pprint(coef_dict)

{100000000.0: 18,
 127427498.57031322: 18,
 162377673.91887242: 18,
 206913808.111479: 18,
 263665089.87303555: 17,
 335981828.6283788: 17,
 428133239.8719396: 17,
 545559478.1168514: 17,
 695192796.1775591: 17,
 885866790.4100832: 16,
 1128837891.6846883: 15,
 1438449888.2876658: 15,
 1832980710.8324375: 13,
 2335721469.0901213: 12,
 2976351441.6313133: 10,
 3792690190.7322536: 6,
 4832930238.571753: 5,
 6158482110.6602545: 3,
 7847599703.514623: 1,
 10000000000.0: 1}


In [18]:
l1_penalty_min = 2976351441.6313128
l1_penalty_max = 3792690190.7322536

# Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set
We will now explore the narrow region of l1_penalty values we found:

In [19]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

For l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
Fit a regression model with a given l1_penalty on TRAIN data. Specify l1_penalty=l1_penalty and l2_penalty=0. in the parameter list. When you call linear_regression.create() make sure you set validation_set = None
Measure the RSS of the learned model on the VALIDATION set
Find the model that the lowest RSS on the VALIDATION set and has sparsity equal to max_nonzero.

In [21]:
validation_rss = {}
for l1_penalty in l1_penalty_values:
    model = tc.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    predictions = model.predict(validation)
    residuals = validation['price'] - predictions
    rss = sum(residuals**2)
    validation_rss[l1_penalty] = rss, model.coefficients['value'].nnz()
    

In [25]:
bestRSS=0

for k,v in validation_rss.items():    
    if (v[1] == max_nonzeros) and (v[0] < bestRSS):
        bestRSS = v[0]
        bestl1 = k
        
print (bestRSS, bestl1)

NameError: name 'bestl1' is not defined